In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

train_set = torchvision.datasets.FashionMNIST('./data', download=True,
                                              train=True,
                                              transform=transform)
train_loader = DataLoader(train_set, batch_size=100)

test_set = torchvision.datasets.FashionMNIST('./data', download=True,
                                             train=False,
                                             transform=transform)
test_loader = DataLoader(test_set, batch_size=100)

writer = SummaryWriter('/content/drive/My Drive/Colab Notebooks/fashionMNIST trial')


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def output_label(label):
    output_mapping = {0: 'T-shirt/Top',
                      1: 'Trouser',
                      2: 'Pullover',
                      3: 'Dress',
                      4: 'Coat',
                      5: 'Sandal',
                      6: 'Shirt',
                      7: 'Sneaker',
                      8: 'Bag',
                      9: 'Ankle Boot'}
    input = (label.item() if type(label) == torch.Tensor
             else label)
    return output_mapping[input]

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc1 = nn.Linear(in_features=64 * 6 * 6, out_features=600)
        # self.drop = nn.Dropout2d(.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        # out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
# device = 'cpu'

In [ ]:
def fit(model, epochs, criterion, optimizer, version, batch_size):
    model.to(device)
    for epoch in range(epochs):
        running_loss = 0
        for i, data in enumerate(train_loader, 0):
            # prog_bar = tqdm(desc='training in progress', total=)
            inputs, input_labels = data
            inputs, input_labels = inputs.to(device), input_labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, input_labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if not (i % batch_size):
                total = 0
                correct = 0
                for tests, test_labels in test_loader:
                    tests, test_labels = tests.to(device), test_labels.to(device)
                    test_outputs = model(tests)
                    # writer.add_embedding(mat=test_outputs,
                    #                      metadata=test_labels,
                    #                      label_img=tests,
                    #                      global_step=1)
                    preds = torch.max(test_outputs, 1)[1]
                    correct += (preds == test_labels).sum()
                    total += len(test_labels)
                accuracy = correct.item() / total * 100
                writer.add_scalar('training loss - version {}'.format(version),
                                  running_loss / batch_size,
                                  epoch * len(train_loader) + i)
                writer.add_scalar('training accuracy - version {}'.format(version),
                                  accuracy,
                                  epoch * len(train_loader) + i)
                print(f'training_loss: {running_loss / batch_size}')
                print(f'training_accuracy: {accuracy}')
                running_loss = 0
    print('Finished training')

In [ ]:
num_epochs = 5

model1 = CNN()
criterion1 = nn.CrossEntropyLoss()
lr1 = .001
optimizer1 = torch.optim.Adam(model1.parameters(), lr=lr1)
print(model1)
fit(model1, num_epochs, criterion1, optimizer1, 1, batch_size=100)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)
training_loss: 0.022865502834320067
training_accuracy: 29.360000000000003
training_loss: 0.6623908415436744
training_accuracy: 80.96
training_loss: 0.422689658254385
training_accuracy: 84.42
training_loss: 0.38502688258886336
training_accur

In [ ]:
PATH = './CNN.pth'
torch.save(model1.state_dict(), PATH)

In [ ]:
%tensorboard --logdir=/content/drive/My Drive/Colab Notebooks/fashionMNIST trial

UsageError: Line magic function `%tensorboard` not found.
